# 来源

** RL **:
    
    介绍：http://karpathy.github.io/2016/05/31/rl/
    
    代码：https://gist.github.com/karpathy/a4166c7fe253700972fcbc77e4ea32c5
    代码keras: https://github.com/keon/policy-gradient/blob/master/pg.py

    中文：https://www.jiqizhixin.com/articles/2016-06-15

** gym教程：**
    
    https://www.cnblogs.com/mandalalala/p/6227201.html

# gym 初步了解

In [2]:
from matplotlib import pyplot as plt
import numpy as np

# plt.ion()
%matplotlib inline
import gym
env = gym.make('Pong-v0')
init_observ = env.reset() #重置环境的状态，返回observations。
print(init_observ.shape)
print(env.action_space) # 查看这个环境中可用的 action 有多少个
print(env.observation_space.shape)    # 查看这个环境中可用的 state 的 observation 有多少个

(210, 160, 3)
Discrete(6)
(210, 160, 3)


In [21]:
print(env.action_space.)

Discrete(6)


In [20]:
# 每一个游戏不同, 每个observation 不同
env = gym.make('CartPole-v0')
print(env.action_space) # 查看这个环境中可用的 action 有多少个
print(env.observation_space.shape)    # 查看这个环境中可用的 state 的 observation 有多少个

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Discrete(2)
(4,)


In [23]:
env = gym.make('CartPole-v0')
for i_episode in range(10): # 玩10次
    observation = env.reset()
    for t in range(100):    # 每次至少100个时间步长，100个时间步长也有可能游戏没有结束哦。。。
        env.render()        # 刷新当前环境，并显示
        print(observation)  # 打印每次的观察值
        action = 1#env.action_space.sample()  # 随机选取一个动作
        observation, reward, done, info = env.step(action) # step 根据下一步的action，返回： 观察，奖励，是否结束一个环节，调试信息。
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[ 0.03371105 -0.04982973  0.00835811  0.0419115 ]
[ 0.03271446  0.14517138  0.00919634 -0.24812269]
[ 0.03561789  0.34016079  0.00423388 -0.53789076]
[ 0.0424211   0.53522296 -0.00652393 -0.82923664]
[ 0.05312556  0.73043349 -0.02310866 -1.12396422]
[ 0.06773423  0.92585062 -0.04558795 -1.423805  ]
[ 0.08625124  1.1215056  -0.07406405 -1.73038077]
[ 0.10868136  1.31739122 -0.10867166 -2.0451586 ]
[ 0.13502918  1.51344811 -0.14957484 -2.36939576]
[ 0.16529814  1.70954852 -0.19696275 -2.70407325]
Episode finished after 10 timesteps
[-0.04881488  0.04344371  0.04580856  0.00554066]
[-0.04794601  0.23787979  0.04591938 -0.2723445 ]
[-0.04318841  0.43231746  0.04047249 -0.55019801]
[-0.03454206  0.62684825  0.02946853 -0.82985959]
[-0.0220051   0.82155525  0.01287134 -1.1131308 ]
[-0.00557399  1.01650583 -0.00939128 -1.40174837]
[ 0.01475613  1.21174319 -0.03742625 -1.69735249]
[ 0.03899099  1

# RL 

In [8]:
""" Trains an agent with (stochastic) Policy Gradients on Pong. Uses OpenAI Gym. """
import numpy as np
import pickle
import gym

# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
resume = True # resume from previous checkpoint?
render = False

# model initialization
D = 80 * 80 # input dimensionality: 80x80 grid
if resume:
    model = pickle.load(open('save.p', 'rb'))
else:
    model = {}
    model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization 权重使用xavier初始化
    model['W2'] = np.random.randn(H) / np.sqrt(H)
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

In [34]:
[_ for _ in reversed([1,2,3])]

[3, 2, 1]

In [2]:
def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
    """ 处理图片,裁剪,下采样,改变大小,变换为0和1"""
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop 
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

def discount_rewards(r):
    # 对于每一步所采样的动作，我们考虑未来所有回报的加权和，而且未来回报的重要性随时间以指数递减
    """ take 1D float array of rewards and compxxxute discounted reward """
    discounted_r = np.zeros_like(r) 
    running_add = 0
    for t in reversed(range(0, r.size)):  
        if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r 

def policy_forward(x):
    """ 前向传播, 两层的策略网络"""
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # return probability of taking action 2, and hidden state

def policy_backward(eph, epdlogp):
    """ backward pass. (eph is array of intermediate hidden states) """
    dW2 = np.dot(eph.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[eph <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    return {'W1':dW1, 'W2':dW2}

In [3]:
np.random.uniform()

0.772770632091281

In [31]:
a=[[1,2],[3,4]]
np.vstack(a)
reversed(range(0, 10))

In [37]:
a = np.array([1,2,3])
b = np.array([4,5,6])
b * a

array([ 4, 10, 18])

In [35]:
env = gym.make("Pong-v0")
observation = env.reset()
prev_x = None # used in computing the difference frame
xs,hs,dlogps,drs = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0
while True:
  if render: env.render()

  # preprocess the observation, set input to network to be difference image. 
  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D) #两帧画面的差
  prev_x = cur_x

  # forward the policy network and sample an action from the returned probability 
  aprob, h = policy_forward(x) #输入到策略网络并得到下一步动作的概率（动作action=2的概率） 
  action = 2 if np.random.uniform() < aprob else 3 # roll the dice! 按照伯努利分布(aprob,1-aprob),选取一个动作

  # record various intermediates (needed later for backprop)
  xs.append(x) # observation
  hs.append(h) # hidden state
  y = 1 if action == 2 else 0 # a "fake label"  # 把所有action==2 的y值为1, 定义这步动作的”实际“标签为1，为什么？
  dlogps.append(y - aprob) #grad that encourages the action that was taken to be taken 
    #(see http://cs231n.github.io/neural-networks-2/#losses if confused)  就是sigmoid 对应 logloss的梯度
    
  # step the environment and get new measurements
  observation, reward, done, info = env.step(action) # 得到其反馈
  reward_sum += reward

  drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

  if done: # an episode finished
    episode_number += 1

    # stack together all inputs, hidden states, action gradients, and rewards for this episode
    epx = np.vstack(xs)
    print(epx.shape,xs[0].shape)
    eph = np.vstack(hs)
    print(eph.shape)
    epdlogp = np.vstack(dlogps)
    epr = np.vstack(drs)
    print(epr.shape,drs[0])
    #print(drs[:30])
    xs,hs,dlogps,drs = [],[],[],[] # reset array memory

    # compute the discounted reward backwards through time
    discounted_epr = discount_rewards(epr) # 计算其所有的回报
    print(discounted_epr.shape)
    #break

    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_epr -= np.mean(discounted_epr)
    discounted_epr /= np.std(discounted_epr)
    print(epdlogp.shape, discounted_epr.shape)
    #break
    epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
    grad = policy_backward(eph, epdlogp)
    for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch # 梯度累加

    # perform rmsprop parameter update every batch_size episodes
    if episode_number % batch_size == 0:
      for k,v in model.items():
        g = grad_buffer[k] # gradient
        rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
        model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
        grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

    # boring book-keeping
    running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
    print('resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward))
    if episode_number % 100 == 0: pickle.dump(model, open('save.p', 'wb'))
    reward_sum = 0
    observation = env.reset() # reset env
    prev_x = None

  if reward != 0: # Pong has either +1 or -1 reward exactly when game ends.
    print(('ep %d: game finished, reward: %f'%(episode_number, reward)) + ('' if reward == -1 else ' !!!!!!!!'))

ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: 1.000000 !!!!!!!!
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
ep 0: game finished, reward: -1.000000
(1623, 6400) (6400,)
(1623, 200)
(1623, 1) 0.0
(1623, 1)
(1623, 1) (1623, 1)
